# Estimation de la couche finale d'Akita sur les données de neurones

In [1]:
import json
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm

In [2]:
import h5py
import cooler

In [3]:
import matplotlib.pyplot as plt

## Chargement des prédictions de l'avant-dernière couche d'Akita

In [8]:
predpath = "/home/bureau/projects/def-bureau/bureau/ran-donnees/PredictNeuronHi-C/akita_pred_sans_final_lisse/"
predfile = predpath + "preds.h5"

In [9]:
pred = h5py.File(predfile, 'r')
pred

<HDF5 file "preds.h5" (mode r)>

In [10]:
pred['preds'].shape

(7619, 99681, 48)

## Chargement des cibles

In [11]:
targetfile = "/home/bureau/projects/def-bureau/bureau/distiller/iPSC/data/1s/seqs_cov/0.h5"

In [12]:
targets = h5py.File(targetfile, 'r')
targets

<HDF5 file "0.h5" (mode r)>

In [13]:
# Les données d'élaboration sont les 7619 premières
train_targets = targets['targets'][:7619,]
train_targets.shape

(7619, 99681)

## Test de la régression linéaire sur un lot

In [14]:
X = np.array(pred['preds'][0,:pred['preds'].shape[1],:pred['preds'].shape[2]],dtype=np.float32)
X.shape

(99681, 48)

In [18]:
X = sm.add_constant(X)
X.shape

(99681, 49)

In [19]:
mod0 = sm.OLS(train_targets[0,:],X)
mod0.fit = mod0.fit()

In [20]:
mod0.fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     1294.
Date:                Mon, 22 Mar 2021   Prob (F-statistic):               0.00
Time:                        17:42:41   Log-Likelihood:                -45880.
No. Observations:               99681   AIC:                         9.186e+04
Df Residuals:                   99632   BIC:                         9.232e+04
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6099      0.019    -32.636      0.000      -0.647      -0.573
x1             0.2036      0.051      4.001      0.000       0.104       0.303
x2             0.2304      0.026      8.868      0.000       0.179       0.281
x3             0.3926      0.027     14.412      0.000       0.339       0.446
x4            -0.6268      0.037    -16.721      0.000      -0.700      -0.553
x5             0.5518      0.043     12.979      0.000       0.468       0.635
x6            -0.0086      0.019     -0.445      0.656      -0.046       0.029
x7             0.4214      0.044      9.586      0.000       0.335       0.508
x8            -0.2067      0.016    -13.012      0.000      -0.238      -0.176
x9             0.5161      0.030     17.013      0.000       0.457       0.576
x10           -0.2607      0.017    -15.731      0.000      -0.293      -0.228
x11           -0.3635      0.026    -13.720      0.000      -0.415      -0.312
x12           -1.3288      0.044    -30.001      0.000      -1.416      -1.242
x13           -0.0365      0.013     -2.775      0.006      -0.062      -0.011
x14           -1.4972      0.039    -38.625      0.000      -1.573      -1.421
x15           -0.2013      0.050     -4.028      0.000      -0.299      -0.103
x16            0.0729      0.032      2.262      0.024       0.010       0.136
x17           -0.8973      0.038    -23.320      0.000      -0.973      -0.822
x18            0.5722      0.048     12.028      0.000       0.479       0.665
x19           -0.2319      0.024     -9.757      0.000      -0.278      -0.185
x20            0.3824      0.018     21.263      0.000       0.347       0.418
x21            0.0708      0.049      1.456      0.145      -0.025       0.166
x22           -0.0715      0.017     -4.297      0.000      -0.104      -0.039
x23           -0.2968      0.048     -6.218      0.000      -0.390      -0.203
x24           -0.1630      0.023     -7.038      0.000      -0.208      -0.118
x25           -0.6044      0.041    -14.752      0.000      -0.685      -0.524
x26            0.0885      0.012      7.505      0.000       0.065       0.112
x27           -0.6791      0.047    -14.450      0.000      -0.771      -0.587
x28           -0.3352      0.025    -13.299      0.000      -0.385      -0.286
x29           -0.0959      0.031     -3.106      0.002      -0.156      -0.035
x30           -1.3957      0.070    -19.816      0.000      -1.534      -1.258
x31           -0.6979      0.038    -18.448      0.000      -0.772      -0.624
x32           -0.0624      0.044     -1.430      0.153      -0.148       0.023
x33            1.1508      0.045     25.698      0.000       1.063       1.239
x34            1.4121      0.060     23.510      0.000       1.294       1.530
x35            0.3260      0.021     15.880      0.000       0.286       0.366
x3

## Régression linéaire pour tous les lots

In [14]:
import multiprocessing as mp
#from joblib import Parallel, delayed

In [15]:
# Nombre de coefficients
npar = pred['preds'].shape[2]+1
beta_mat = np.zeros((pred['preds'].shape[0],npar))
xx_mat = np.zeros((pred['preds'].shape[0],npar,npar))
xx_sum = np.zeros((npar,npar))
inputs=range(pred['preds'].shape[0])
 
def compute_LM(i):
    # Extraction de la matrice de prédicteurs
    X = np.array(pred['preds'][i,:pred['preds'].shape[1],:pred['preds'].shape[2]],dtype=np.float32)
    X = sm.add_constant(X)
    mod = sm.OLS(train_targets[i,:],X)
    mod.fit = mod.fit()
    return(mod.fit.params,np.linalg.inv(mod.fit.normalized_cov_params))

In [16]:
# processeurs
ncpus = int(os.environ.get('SLURM_CPUS_PER_TASK',default=1))
pool = mp.Pool(processes=ncpus)
ncpus

8

In [17]:
# Boucle sur les lots
#processed_list = Parallel(n_jobs=num_cores)(delayed(compute_LM)(i=j)for j in inputs)
processed_list = [pool.apply_async(compute_LM,args=(j,)) for j in inputs]

In [19]:
# Traitement des sorties
beta_vec = np.zeros(npar)
for i in range(len(processed_list)):
    w_s = processed_list[i].get()
    beta_mat[i,] = w_s[0]
    xx_mat[i,] = w_s[1]
    xx_sum = xx_sum + xx_mat[i,]
    beta_vec = beta_vec + np.matmul(xx_mat[i,],beta_mat[i,]) 

In [ ]:
# Sauvegarde des coefficients et matrices X'X dans un fichier h5
hf = h5py.File('regress_res.h5', 'w')
hf.create_dataset('beta',data=beta_mat)
hf.create_dataset('xx',data=xx_mat)
hf.close()

In [20]:
# Inversion de la somme des matrices de variance-covariance
cov = np.linalg.inv(xx_sum)
cov.shape

(49, 49)

## Calcul de l'estimation des moindre carrés des coefficients par la méthode de Duncan (1980)

In [21]:
beta_final = np.matmul(cov,beta_vec)
beta_final

array([-1.51718975e-01,  2.58807564e-02,  1.19163258e-02,  2.93938664e-03,
       -6.61203015e-02, -9.59953129e-03,  2.14761907e-02, -1.19366767e-02,
       -1.60369408e-02, -1.61147827e-02,  8.02132985e-03,  2.82380447e-03,
        8.35432573e-02, -2.65990678e-02, -6.22075246e-02, -8.04181107e-02,
        3.63725560e-02,  6.38940941e-02,  7.72009556e-02,  5.82915440e-02,
       -5.51047975e-02, -1.85974530e-02, -1.36052504e-02,  5.71852089e-03,
       -4.88973728e-02,  3.70050306e-02,  1.69409949e-02, -6.48172132e-02,
        1.16648122e-02, -6.57552656e-04, -1.15045980e-01,  4.25135569e-02,
        3.24065033e-02, -1.53930364e-01,  2.30432403e-02,  1.79276000e-02,
        2.47894460e-02,  3.85482915e-03, -3.03972085e-02,  6.91088834e-03,
       -7.33670091e-02, -5.59882123e-02,  1.22114142e-04, -1.38735699e-02,
        8.35359012e-03,  4.15396286e-02,  7.20825595e-02, -2.02940561e-02,
        1.50881877e-02])

## Sauvegarde des estimations et de leur "covariance" dans des fichiers

In [38]:
beta_dat = pd.DataFrame(beta_final)
beta_dat.to_csv("beta_final.csv")

In [40]:
cov_dat = pd.DataFrame(cov)
cov_dat.to_csv("cov_final.csv")

In [22]:
cov[:5,:5]

array([[ 7.81481137e-08, -4.12235824e-08,  1.41750080e-08,
        -3.32145501e-09, -3.71675485e-08],
       [-4.12235824e-08,  9.49145029e-07, -4.95143495e-08,
        -1.86696885e-07,  3.89394626e-08],
       [ 1.41750080e-08, -4.95143495e-08,  2.92871352e-07,
         2.32118580e-08, -1.63626059e-08],
       [-3.32145501e-09, -1.86696885e-07,  2.32118580e-08,
         3.16309781e-07, -2.57895686e-08],
       [-3.71675485e-08,  3.89394626e-08, -1.63626059e-08,
        -2.57895686e-08,  3.44028424e-07]])

In [23]:
beta_mat[1000,]

array([-0.39902382, -1.58300518,  0.04490336,  0.19111755,  1.38983807,
       -0.11763738, -0.61290574,  0.38687857,  0.54073968,  0.97442179,
       -0.14012155,  1.03036003, -0.45871646,  0.76067161,  0.492336  ,
        2.27783643,  0.82602485, -1.61935474, -0.00281972, -0.44336617,
        0.34788088, -0.27392429, -0.11659872, -0.82399811,  0.31070679,
        0.2155158 , -0.17044884,  0.30385767, -0.65210356,  0.04488354,
        0.55268289,  1.1322074 , -0.50240073,  2.04427803,  0.15309189,
       -0.46671125,  1.00288281, -0.25352349,  0.89545993,  0.28146678,
       -1.0748859 , -0.4149236 , -0.09360222,  0.16673444, -0.75240393,
       -1.21771065, -1.31073787, -0.15942402, -0.39437459])